In [ ]:
import sys
sys.path.append('..')

from utils import show
import image_processes as imgp
import map_processes as mapp
import utils as ut
import numpy as np
import cv2
import os
import numpy as np
from skimage.morphology import medial_axis
from skimage.util import invert
from skimage.io import imread
import json

%load_ext autoreload
%autoreload 2


In [ ]:
imgName = 'map.png'
texturePath = '../../../textures/map512/'
outputPath = './outputs/thickening/'

imgPath = os.path.join(texturePath, imgName)

image = imread(imgPath, as_gray=True)

height, width = image.shape
# imgSize = (250, 250)
imgSize = (width, height)
image = cv2.resize(image, imgSize)

if not os.path.exists(outputPath):
    os.makedirs(outputPath)

# Copy imgPath as input.png
inputPath = os.path.join(outputPath, 'input.png')
cv2.imwrite(inputPath, cv2.cvtColor(imread(imgPath), cv2.COLOR_BGR2RGB))


In [ ]:
# ut.show((image * 255).astype(np.uint8))

In [ ]:
# Skeletonize
skeleton = imgp.skeletonize(image, 0.35, True, False)
skeleton_uint8 = (skeleton * 255).astype(np.uint8)
ut.saveCVImage(outputPath, 'skeleton.png', skeleton_uint8)

In [ ]:
# DILATATION
imgDilated = imgp.dilatation(skeleton_uint8, 4, True)
ut.saveCVImage(outputPath, 'simpleDilatation.png', imgDilated)

In [ ]:
# Blur
imgBlurred = imgp.blur(skeleton_uint8, 6, True)
ut.saveCVImage(outputPath, 'simpleBlur.png', imgBlurred)


In [ ]:
# Funny use of DILATATION to remove borders
imgp.removeLines(imread(imgPath), 5, False)

In [ ]:
# Test superposition of dilatations
composition = imgp.test(skeleton, 4, False)
cv2.imwrite(os.path.join(outputPath, 'composition.png'), composition);

In [ ]:
# Normalize blurred image
blur = imgp.blur(skeleton_uint8, 10, False)
blur_rgba = imgp.normalizeBlurredImage(blur, 80, True)
cv2.imwrite(os.path.join(outputPath, 'blurred.png'), blur_rgba)


In [ ]:
# Thickening
color = imgp.hex_to_rgba("#614338")
normalizedBlurredDilatedImg = imgp.thickening(
    skeleton, 8, 4, 80, color, True, False)
ut.saveCVImage(outputPath, 'thickening.png',
               normalizedBlurredDilatedImg)